In [24]:
%pip install keras

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
jupyter-server 1.13.5 requires pywinpty<2; os_name == "nt", but you have pywinpty 2.0.2 which is incompatible.



  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.11.2
    Uninstalling Pygments-2.11.2:
      Successfully uninstalled Pygments-2.11.2


In [27]:
%pip install optree


Note: you may need to restart the kernel to use updated packages.


In [34]:
%pip install --upgrade typing-extensions


Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: HTTPSConnectionPool(host='files.pythonhosted.org', port=443): Max retries exceeded with url: /packages/b4/71/1ac7960071a524fa25d7e9dd5040da26d3e15802d869c588d7e4cd8036f9/tensorflow-2.17.0-cp39-cp39-win_amd64.whl (Caused by NewConnectionError('<pip._vendor.urllib3.connection.HTTPSConnection object at 0x00000239E101EC40>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))



In [24]:

from submit import my_fit, my_predict
import time as tm
import pickle
import warnings
import os

In [25]:
with open( "dict", 'r' ) as f:
	words_train = f.read().split( '\n' )[:-1]		# Omit the last line since it is empty
	num_words = len( words_train )
 

In [26]:
with open( "dummy/dict_secret", 'r' ) as f:
	words_test = f.read().split( '\n' )[:-1]		# Omit the last line since it is empty
	num_words = len( words_test )
 

In [27]:
n_trials = 5

t_train = 0
m_size = 0
t_test = 0
prec = 0

In [28]:
def get_bigrams( word, lim = None ):
  # Get all bigrams
  bg = map( ''.join, list( zip( word, word[1:] ) ) )
  # Remove duplicates and sort them
  bg = sorted( set( bg ) )
  # Make them into an immutable tuple and retain only the first few
  return tuple( bg )[:lim]

In [29]:
lim_bg = 5
lim_out = 5

for t in range(n_trials):
    tic = tm.perf_counter()
    model = my_fit(words_train)
    toc = tm.perf_counter()
    t_train += toc - tic

    with open(f"model_dump_{t}.pkl", "wb") as outfile:
        pickle.dump(model, outfile, protocol=pickle.HIGHEST_PROTOCOL)

    m_size += os.path.getsize(f"model_dump_{t}.pkl")

    tic = tm.perf_counter()

    for (i, word) in enumerate(words_test):
        bg = get_bigrams(word, lim=lim_bg)
        guess_list = my_predict(model, bg)

        # Do not send long guess lists -- they will result in lower marks
        guess_len = len(guess_list)
        # Ignore all but the first 5 guesses
        guess_list = guess_list[:lim_out]

        # Notice that if 10 guesses are made, one of which is correct,
        # score goes up by 1/10 even though only first 5 guesses are considered
        # Thus, it is never beneficial to send more than 5 guesses
        if word in guess_list:
            prec += 1 / guess_len

    toc = tm.perf_counter()
    t_test += toc - tic



In [30]:
t_train /= n_trials
m_size /= n_trials
t_test /= n_trials
prec /= ( n_trials * num_words )

print( t_train, m_size, prec, t_test  )

5.337200700000016 417337732.0 0.9750338687826592 0.9234471000000439
